A Look Into The Top 100 Songs on Billboard (Week of Augest 4th, 2025)

In [1]:
import pandas as pd 
import numpy as np

data = pd.read_csv('../billboard_hot_100_with_lyrics.csv')
data.dtypes

title              object
artist             object
lyrics             object
rank                int64
last_week           int64
peak_rank           int64
weeks_on_chart      int64
release_year      float64
lyric_length        int64
dtype: object

In [2]:
data.head()

,title,artist,lyrics,rank,last_week,peak_rank,weeks_on_chart,release_year,lyric_length
0,Ordinary,Alex Warren,44 ContributorsTranslationsItalianoEspañolУкра...,1,1,1,25,2025.0,356
1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI",53 ContributorsTranslationsRomanizationEnglish...,2,2,2,6,2025.0,345
2,What I Want,Morgan Wallen Featuring Tate McRae,22 ContributorsTranslationsFrançaisItalianoNed...,3,4,1,11,2025.0,433
3,Daisies,Justin Bieber,44 ContributorsTranslationsČeskyItalianoPolski...,4,3,2,3,2025.0,341
4,Just In Case,Morgan Wallen,17 ContributorsTranslationsEspañolJust In Case...,5,5,2,19,2025.0,391


Data Cleaning

In [3]:
import re

def clean_lyrics(lyrics):
    
    if pd.isna(lyrics) or lyrics == '':
        return None
    
    # Convert lyrics to string type (allows for slicing but intuitively makes sense for lyrics)
    lyrics = str(lyrics)
    
    # Regex expression gets everything before and including the first 'lyrics' (used to remove the first line)
    lyrics_match = re.search(r'.*?lyrics\s*', lyrics, re.IGNORECASE)
    if lyrics_match:
        lyrics = lyrics[lyrics_match.end():]
        
    # Same as above but include | (bitwise OR) re.DOTALL (to match across multiple lines)    
    read_more_match = re.search(r'.*?read more\s*', lyrics, re.IGNORECASE | re.DOTALL)
    if read_more_match:
        lyrics = lyrics[read_more_match.end():]
        
        
    lyrics = re.sub(r'[""]', '"', lyrics)  # Normalize quotes
    lyrics = re.sub(r'\s+', ' ', lyrics)   # Multiple spaces → single space

    lyrics = lyrics.strip()
    
    return lyrics

print(f"First 200 characters of the first song: \n{data['lyrics'][0][0:200]}\n")

data['lyrics'] = data['lyrics'].apply(clean_lyrics)

print(f"First full song: \n{data['lyrics'][0]}")

First 200 characters of the first song: 
44 ContributorsTranslationsItalianoEspañolУкраїнськаDeutschРусский (Russian)PortuguêsČeskyOrdinary Lyrics
They say, "The holy water's watered down
And this town's lost its faith
Our colors will fade e

First full song: 
They say, "The holy water's watered down And this town's lost its faith Our colors will fade eventually" So, if our time is runnin' out Day after day We'll make the mundane our masterpiece Oh, my, my Oh, my, my love I take one look at you You're takin' me out of the ordinary I want you layin' me down 'til we're dead and buried On the edge of your knife, stayin' drunk on your vine The angels up in the clouds are jealous knowin' we found Somethin' so out of the ordinary You got me kissin' thе ground of your sanctuary Shatter me with your touch, oh, Lord, return mе to dust The angels up in the clouds are jealous knowin' we found Hopeless hallelujah On this side of Heaven's gate Oh, my life, how do ya Breathe and take my breath away?

In [4]:
# Identify songs that will be removed
songs_to_remove = data[data['lyrics'].isna()]
print(f"Songs that will be removed (no lyrics after cleaning):")
if len(songs_to_remove) > 0:
    for index, song in songs_to_remove.iterrows():
        print(f" - '{song['title']}' by {song['artist']} (Rank #{song['rank']})")

# Remove songs without lyrics
print(f"\nSongs before removing null lyrics: {len(data)}")
data = data.dropna(subset=['lyrics'])
print(f"Songs after removing null lyrics: {len(data)}")

Songs that will be removed (no lyrics after cleaning):
 - 'Takedown' by JEONGYEON, JIHYO & CHAEYOUNG Of TWICE (Rank #67)

Songs before removing null lyrics: 100
Songs after removing null lyrics: 99


In [5]:
# Convert title, artist, and lyrics to string (though lyrics should already be string after our cleaning)
data['title'] = data['title'].astype('string')
data['artist'] = data['artist'].astype('string')
data['lyrics'] = data['lyrics'].astype('string')

data['release_year'] = data['release_year'].astype('int64')

# Add a new column for word count
data['word_count'] = data['lyrics'].apply(lambda x: len(x.split()))

data.dtypes

title             string[python]
artist            string[python]
lyrics            string[python]
rank                       int64
last_week                  int64
peak_rank                  int64
weeks_on_chart             int64
release_year               int64
lyric_length               int64
word_count                 int64
dtype: object

In [6]:
data.isna().sum()

title             0
artist            0
lyrics            0
rank              0
last_week         0
peak_rank         0
weeks_on_chart    0
release_year      0
lyric_length      0
word_count        0
dtype: int64

In [7]:
data.to_csv('../billboard_hot_100_cleaned.csv', index=False)

Sentiment

In [8]:
from transformers import pipeline

# Initialize the emotion classifier pipeline with a pretrained model
emotion_classifier = pipeline(
    "text-classification", 
    model="j-hartmann/emotion-english-distilroberta-base",
    return_all_scores=True,
    device=-1
)

print(emotion_classifier("I love programming!")) #testing

c:\Users\danny\Coding Outside\Projects\LyricPopularity\.venv312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu
c:\Users\danny\Coding Outside\Projects\LyricPopularity\.venv312\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'anger', 'score': 0.0026277850847691298}, {'label': 'disgust', 'score': 0.0007785824709571898}, {'label': 'fear', 'score': 0.0003582449571695179}, {'label': 'joy', 'score': 0.9866405129432678}, {'label': 'neutral', 'score': 0.0027272517327219248}, {'label': 'sadness', 'score': 0.003636218374595046}, {'label': 'surprise', 'score': 0.0032313845586031675}]]


In [9]:
def classify_emotions(lyrics):
    
    try:
        
        chunks = [lyrics[i:i+512] for i in range(0, len(lyrics), 512)]
        
        all_emotions_scores = []
        chunk_lengths = []
                    
        emotion_names = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

        for chunk in chunks:
            emotions = emotion_classifier(
                lyrics,
                truncation=True,
                max_length=512
            )[0] # Utilize the pipeline to classify emotions
            emotions_dict = {emotion['label']: emotion['score'] for emotion in emotions} # Create a dictionary from the list of emotions
            
            # If there are missing emotions, they are assigned a score of 0.0
            for emotion in emotion_names:
                if emotion not in emotions_dict:
                    emotions_dict[emotion] = 0.0
            
            all_emotions_scores.append(emotions_dict)
            chunk_lengths.append(len(chunk))
            
        avg_emotions = {}
        for emotion in emotion_names:
            avg_emotions
            weighted_sum = 0.0
            for i in range(len(all_emotions_scores)):
                weighted_sum += all_emotions_scores[i][emotion] * chunk_lengths[i]
            avg_score = weighted_sum / sum(chunk_lengths)
            avg_emotions[emotion] = avg_score
                
        top_emotion = max(avg_emotions.items(), key=lambda x: x[1])
        
        result = {  
            'top_emotion': top_emotion[0],
            'top_emotion_score': top_emotion[1],
            **avg_emotions
        }
        
        return result
    
    except Exception as e:
        print(f"Error classifying emotions for lyrics: {e}")
        return {
            'top_emotion': None,
            'top_emotion_score': None,
            'anger': 0.0,
            'disgust': 0.0,
            'fear': 0.0,
            'joy': 0.0,
            'neutral': 0.0,
            'sadness': 0.0,
            'surprise': 0.0
        }

In [10]:
emotion_results = []

for index, lyrics in enumerate(data['lyrics']):
    if index % 10 == 0 or index == 99:
        print(f"Processing song {index + 1}/{len(data)}")
    emotions = classify_emotions(lyrics[:512])
    emotion_results.append(emotions)
    
emotions_data = pd.DataFrame(emotion_results)
emotions_data.head()
data = pd.concat([data.reset_index(drop=True), emotions_data.reset_index(drop=True)], axis=1)

Processing song 1/99
Processing song 11/99
Processing song 21/99
Processing song 31/99
Processing song 41/99
Processing song 51/99
Processing song 61/99
Processing song 71/99
Processing song 81/99
Processing song 91/99


In [11]:
data

,title,artist,lyrics,rank,last_week,peak_rank,weeks_on_chart,release_year,lyric_length,word_count,top_emotion,top_emotion_score,anger,disgust,fear,joy,neutral,sadness,surprise
0,Ordinary,Alex Warren,"They say, ""The holy water's watered down And t...",1,1,1,25,2025,356,352,sadness,0.354639,0.079038,0.093723,0.091241,0.036207,0.134498,0.354639,0.210655
1,Golden,"HUNTR/X: EJAE, Audrey Nuna & REI AMI","I was a ghost, I was alone 어두워진 앞길속에 (Hah) Giv...",2,2,2,6,2025,345,304,fear,0.383440,0.063155,0.013784,0.383440,0.026303,0.099525,0.206061,0.207732
2,What I Want,Morgan Wallen Featuring Tate McRae,"She said, ""You don't want this heart, boy, it'...",3,4,1,11,2025,433,428,anger,0.451607,0.451607,0.059232,0.230261,0.021143,0.129680,0.091698,0.016379
3,Daisies,Justin Bieber,"Throwin' petals like, ""Do you love me or not?""...",4,3,2,3,2025,341,336,neutral,0.433261,0.324150,0.110138,0.069871,0.014375,0.433261,0.031724,0.016482
4,Just In Case,Morgan Wallen,I ain't sayin' that I always sleep alone I ain...,5,5,2,19,2025,391,386,sadness,0.442795,0.040187,0.021925,0.038544,0.044610,0.391176,0.442795,0.020762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Madeline,Zach Bryan Featuring Gabriella Rose,"(Ready? Yes) Well, oh, Madeline, how you been?...",96,62,62,2,2025,213,210,sadness,0.617379,0.013937,0.018481,0.144597,0.013042,0.161412,0.617379,0.031152
95,Mommanem,"Tyler, the Creator","Bro, bro, I'm in my feelings Bro, huh That ain...",97,54,54,2,2025,261,258,anger,0.632238,0.632238,0.102488,0.100486,0.013145,0.109545,0.026578,0.015519
96,River Washed Hair,Zach Bryan,Remember when we'd sit around an ol' fire And ...,98,90,53,4,2025,364,359,sadness,0.892292,0.016890,0.016956,0.018261,0.004902,0.037540,0.892292,0.013159
97,Don't Tap That Glass / Tweakin',"Tyler, the Creator","Ayy Huh, uh Ayy, uh, ayy, uh, ayy, uh Don't ta...",99,56,56,2,2025,705,697,fear,0.621353,0.144379,0.106711,0.621353,0.007863,0.070027,0.031922,0.017744
